### 💻 **<i>Notebook 03:</i> Conversational Agent and Tools**

**For running on Google Colab only, run the code cell below:**

In [ ]:
import os
from dotenv import load_dotenv
try:
    # This import ONLY works on the Colab web interface
    from google.colab import userdata

    print("Loading API keys from Colab Secrets...")

    # --- Key Retrieval and Injection into os.environ ---
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')
    os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
    os.environ['LANGCHAIN_PROJECT'] = "retrogaming-qa-bot"

    # --- Verification ---
    if not os.environ.get('OPENAI_API_KEY'):
        raise ValueError("OPENAI_API_KEY was not found in Colab Secrets.")

    print("✅ All API keys were successfully loaded from Colab Secrets.")

except ImportError:
    # Fallback for local environments (local VS Code)
    print("Loading API keys from local .env file (VS Code/Local Mode).")
    load_dotenv()

# --- Common Configurations ---
CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

# Here, the rest of the notebook variables are defined (llm, embeddings, etc.)
# ... (should continue defining LLM/RAG configuration variables)

A carregar chaves API do Colab Secrets...
✅ Todas as chaves API foram carregadas com sucesso a partir do Colab Secrets.


##### **00 - Configuration and Imports**

In [2]:
import os
from dotenv import load_dotenv

In [4]:
pip install langchain_openai langchain_community langchain_classic langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
# LangChain Core Imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool

In [4]:
# --- NEW: Imports for Web Search Tool ---
from langchain_community.tools import GoogleSearchRun
from langchain_community.utilities import GoogleSearchAPIWrapper

In [5]:
# --- Legacy Imports for Agent and Memory ---
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.agents import initialize_agent, AgentType
from langchain_classic.memory import ConversationBufferWindowMemory

In [6]:
# --- 00 - Configuration and Imports ---
load_dotenv()

CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

llm = ChatOpenAI(model=LLM_MODEL, temperature=0.0)
print("✅ Configuration loaded. Ready to proceed.")

✅ Configuration loaded. Ready to proceed.


##### **01 - Load Vector Store and Hybrid Tools**

In [7]:
!pip install chromadb
import chromadb
# ====================================================================
# 01 - Load Vector Store and Define Hybrid Tools
# ====================================================================

# 1. Load the existing vector store
print(f"\n[1/4] Loading vector store from: {CHROMA_PATH}...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

try:
    vector_db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )
    print("✅ Successfully loaded vector store.")
except Exception as e:
    print(f"❌ Error loading ChromaDB: {e}.")

# 2. Define the RAG Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})
print(f"✅ RAG Retriever created, set to retrieve top 3 relevant chunks.")

# 3. Define the RAG Wrapper function
def get_retrogaming_docs(query: str) -> str:
    """
    Searches the retrogaming knowledge base for relevant documents based on the query.
    Returns the documents formatted as a string, including source type (YouTube/Web).
    """
    docs = retriever.invoke(query)

    formatted_docs = []
    for doc in docs:
        source_type = doc.metadata.get('source_type', 'unknown')
        formatted_docs.append(f"[{source_type.upper()}] {doc.page_content}")

    return "\n\n".join(formatted_docs)

# 4. Define the RAG Tool (Internal Knowledge)
retrogaming_qa_tool = Tool(
    name="retrogaming_knowledge_base",
    func=get_retrogaming_docs,
    description="Tool to search and retrieve specific information from the internal knowledge base (YouTube transcripts or Web tutorials) about emulators, settings, and consoles. Use this FIRST."
)

# 5. Define the Google Search Tool (External Knowledge)
# 🚨 FIX: Pass the wrapper instance directly to the api_wrapper parameter
search_wrapper = GoogleSearchAPIWrapper()
google_search_tool = GoogleSearchRun(
    name="google_search",
    api_wrapper=search_wrapper, # <--- CORRECTED ARGUMENT
    description="Use this tool to search the internet for current or highly specific information when the internal knowledge base does not have the answer. Useful for new game specs or up-to-date emulator news."
)

# 6. Final List of Tools (Hybrid Agent)
tools = [retrogaming_qa_tool, google_search_tool]
print(f"✅ Hybrid Tools defined: {', '.join([t.name for t in tools])}")


[1/4] Loading vector store from: ./chroma_db_retrogaming...
✅ Successfully loaded vector store.
✅ RAG Retriever created, set to retrieve top 3 relevant chunks.
✅ Hybrid Tools defined: retrogaming_knowledge_base, google_search


/tmp/ipython-input-4231127926.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(
/tmp/ipython-input-4231127926.py:48: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the `langchain-google-community package and should be used instead. To use it run `pip install -U `langchain-google-community` and import as `from `langchain_google_community import GoogleSearchAPIWrapper``.
  search_wrapper = GoogleSearchAPIWrapper()
/tmp/ipython-input-4231127926.py:49: LangChainDeprecationWarning: The class `GoogleSearchRun` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated v

##### **02 - Define the Conversational Prompt and Hybrid Agent**

In [8]:
# ====================================================================
# 02 - Define the Conversational Prompt and Hybrid Agent
# ====================================================================

# The detailed System Prompt is critical for guiding the hybrid agent's behavior.
SYSTEM_TEMPLATE = (
    "You are a comprehensive expert retrogaming assistant and tuning specialist. Your expertise covers "
    "tuning old video games from various platforms (including PC) for better resolution and modern "
    "compatibility on PC, Mac, or the original native console hardware. "

    "***INITIAL CONVERSATION GUIDANCE:***"

    "1. **PC/Mac Path Flow:** If the user chooses PC or Mac:"
    "   - **Gather Specs:** IMMEDIATELY ask for the user's current **computer specs** (CPU/GPU/RAM) and their **monitor's native resolution**. **CRITICAL RULE:** If the user states they do not know how to find their computer specs, you MUST provide clear, step-by-step answers on how to find this information on their operating system (Windows or macOS)."
    "   - **Advice:** Once specs are known, use your tools to provide information on the available **emulators** (like PCSX2, Dolphin, etc.) and **tuning specs** tailored to their hardware for the specific game (e.g., Jak 2 Renegade)."

    "2. **Native Console Path Flow:** If the user chooses the native console:"
    "   - **Advice:** Use your tools to reply with the best options to output that system to its best possible resolution, naming **external third-party devices** like HDMI adapters or image/signal converters."

    "***HYBRID CONSTRAINT (RAG & SEARCH PRIORITY):***"
    "3. **PRIORITY:** You **MUST** attempt to use the tool **'retrogaming_knowledge_base'** first."
    "4. **FALLBACK:** If the required information is **NOT** found in the RAG context, you **MUST** use the tool **'google_search'** to find a reliable external source before formulating the final answer."
    "5. **TRANSPARENCY:** If the final answer is based on the Google search, briefly mention that the information was retrieved from an updated search."

    "\n\nChat History: {chat_history}"
    "\n\nUser Question: {input}"
)


# 1. Define Memory (stores the conversation history)
conversational_memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

# 2. Define the Agent Executor
agent_executor = initialize_agent(
    tools, # RAG Tool + Google Search Tool
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, # Ideal for tools and memory
    verbose=True,
    memory=conversational_memory,
    agent_kwargs={"system_message": SYSTEM_TEMPLATE}
)

print("\n[3/4] ✅ Hybrid Agent Executor (RAG + Google Search) created.")


[3/4] ✅ Hybrid Agent Executor (RAG + Google Search) created.


/tmp/ipython-input-205781746.py:31: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferWindowMemory(
/tmp/ipython-input-205781746.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent_executor = initialize_agent(


##### **03 - Test the Conversational Agent**

In [9]:
# ====================================================================
# 03 - Test the Conversational Agent
# ====================================================================

# NOTE: The MANDATORY STARTING QUESTION is handled in Notebook 04 (Gradio).
# We simulate Turn 2 onward here, assuming the user already chose "PC/Mac".

# 1. Start the conversation (User asks for the game, ignoring the mandatory question for this test)
print("\n--- Conversation Start (Turn 1: User asks for game) ---")
result_1 = agent_executor.invoke({"input": "I want to play an old game called 'Jak 2 Renegade'."})
print(f"\nAgent: {result_1['output']}")

# 2. Respond (Agent should now enforce the flow: ask for platform, even if the user ignored it in the input)
print("\n--- Conversation Response (Turn 2: Agent enforces flow) ---")
result_2 = agent_executor.invoke({"input": "I choose PC/Mac, but I don't know my computer specs."})
print(f"\nAgent: {result_2['output']}")

# 3. Final Answer (User gives specs, Agent should now use tools to find emulator/specs)
print("\n--- Conversation Response (Turn 3: Agent uses tools) ---")
result_3 = agent_executor.invoke({"input": "My specs are i7 12th gen, 32GB RAM, and my monitor is 4K. I will use a controller."})
print(f"\nAgent: {result_3['output']}")

print("\n[4/4] ✅ Testing complete. Proceed to Notebook 04 for deployment.")


--- Conversation Start (Turn 1: User asks for game) ---


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Search for information about 'Jak 2 Renegade'
Observation: 
Thought:Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Search for information about 'Jak 2 Renegade'
Observation: 
Thought: Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Search for information about 'Jak 2 Renegade'
Observation: 
Thought: Do I need to use a tool? No
AI: Jak 2: Renegade is a popular action-adventure game released for the PlayStation 2. It is the second game in the Jak and Daxter series developed by Naughty Dog. The game follows the story of Jak and his friends as they navigate through a dystopian city and fight against various enemies. If you're looking to play Jak 2: Renegade, you may want to consider using a PlayStation 2 emulator on your computer to experience the g